# 주식가격 예측 데이터 분석 모델링 부분 코드

In [20]:
#코스닥 추출
kosdaq_list=stock_list.loc[stock_list.상장시장=='KOSDAQ']
kospi_list=stock_list.loc[stock_list.상장시장=='KOSPI']

# random forest 주가 변화율 예측
## 다음주의 평균가격을 예측해보기

In [26]:
#자체적으로 결과값과 예측값을 비교하기위해 결과셋 생성 
start_date_private='20211129' 
end_date_private='20211203' 
y_private=pd.DataFrame(columns=[0,1,2,3,4]) 
for i in tqdm(list(stock_list.종목코드)):
    sample = fdr.DataReader(i, start = start_date_private, end = end_date_private)[['Close']].reset_index() 
    y_private.loc[i,:]=list(sample.Close)
y_private.head()

100%|████████████████████████████████████████████████████████████████████████████████| 343/343 [00:39<00:00,  8.60it/s]


,0,1,2,3,4
000070,95900,91100,95700,98000,98900
000080,30300,29000,29400,29550,30600
000100,56369,55317,55221,57518,57709
000120,129000,124000,125000,129000,131000
000150,110500,108500,112000,110000,108500


# 위 parameter 기반 전체 모델링

# default model

In [135]:
#나스닥 코스피 지수 추출
start_date='20201224'
end_date='20211129'
nasdaq=pdr.get_data_yahoo('^IXIC',start_date,end_date)
nasdaq['nasdaq']=nasdaq['Close']
kospi=pdr.get_data_yahoo('^KS11',start_date,end_date)
kospi['kospi']=kospi.Close
kosdaq=pdr.get_data_yahoo('^KQ11',start_date,end_date)
kosdaq['kosdaq']=kosdaq.Close

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [136]:
#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(random_state=123)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill') #나스닥 추가
    train1=pd.concat([train1,kospi['kospi']],axis=1) #코스피 추가
    train1=pd.concat([train1,kosdaq['kosdaq']],axis=1) #코스닥 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

  0%|                                                                                          | 0/343 [00:00<?, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

  2%|█▋                                                                                | 7/343 [00:01<01:27,  3.86it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

  4%|███▎                                                                             | 14/343 [00:03<01:27,  3.76it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

  6%|████▉                                                                            | 21/343 [00:05<01:31,  3.53it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

  8%|██████▌                                                                          | 28/343 [00:07<01:23,  3.75it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

...

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 91%|████████████████████████████████████████████████████████████████████████▊       | 312/343 [01:19<00:06,  4.63it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: Set

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 93%|██████████████████████████████████████████████████████████████████████████▏     | 318/343 [01:20<00:05,  4.66it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:

 94%|███████████████████████████████████████████████████████████████████████████▌    | 324/343 [01:22<00:04,  4.54it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 97%|█████████████████████████████████████████████████████████████████████████████▏  | 331/343 [01:23<00:02,  4.77it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: Set

 98%|██████████████████████████████████████████████████████████████████████████████▌ | 337/343 [01:24<00:01,  4.95it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3849879473.py:24: SettingWithCopyWarning: 
A 

In [137]:
result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction)
(NMAE(y_private.reset_index(drop=True),result0)*100).mean()

C:\Users\kimpu\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3417: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


3.6771214906124947

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

MAE =  mean_absolute_error(y_private.reset_index(drop=True), result0)
# MAE = 0.45454545454545453

MSE = mean_squared_error(y_private.reset_index(drop=True), result0)
# MSE = 0.45454545454545453

RMSE = np.sqrt(MSE)
# RMSE = 0.674199862463242

MSLE = mean_squared_log_error(y_private.reset_index(drop=True), result0)
# MSLE = 0.029272467607503516

RMSLE = np.sqrt(mean_squared_log_error(y_private.reset_index(drop=True), result0))
# RMSLE = 0.1710919858073531

R2 = r2_score(y_private.reset_index(drop=True), result0)
# R2 = 0.868421052631579

In [45]:
prediction.shape

(343,)

In [46]:
result0.to_numpy().reshape(343*5,1)

array([[ 96760.8  ],
       [ 96760.8  ],
       [ 96760.8  ],
       ...,
       [264215.196],
       [264215.196],
       [264215.196]])

In [47]:
y_private.to_numpy().reshape(343*5,1)

array([[95900],
       [91100],
       [95700],
       ...,
       [264200],
       [274700],
       [275900]], dtype=object)

In [48]:
mean_squared_error(y_private.to_numpy().reshape(343*5,1), result0.to_numpy().reshape(343*5,1))

46967564.94933186

In [33]:
prediction2=model.predict(x_train)

In [34]:
prediction2.shape

(75460,)

In [38]:
MSE

46967564.94933186

In [39]:
#1최근 데이터로 예측해보기
#나스닥 코스피 지수 추출
start_date='20211120'
end_date='20221217'
nasdaq=pdr.get_data_yahoo('^IXIC',start_date,end_date)
nasdaq['nasdaq']=nasdaq['Close']
kospi=pdr.get_data_yahoo('^KS11',start_date,end_date)
kospi['kospi']=kospi.Close
kosdaq=pdr.get_data_yahoo('^KQ11',start_date,end_date)
kosdaq['kosdaq']=kosdaq.Close

model=RandomForestRegressor(max_features=2,n_estimators=300,random_state=123)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill') #나스닥 추가
    train1=pd.concat([train1,kospi['kospi']],axis=1) #코스피 추가
    train1=pd.concat([train1,kosdaq['kosdaq']],axis=1) #코스닥 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)
x_private

  0%|                                                                                          | 0/343 [00:00<?, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3362256002.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3362256002.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3362256002.py:32: SettingWithCopyWarning: 
A 

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
start_date_private='20211213' 
end_date_private='20211217' 
y_private00=pd.DataFrame(columns=[0,1,2,3,4]) 
for i in tqdm(list(stock_list.종목코드)):
    sample = fdr.DataReader(i, start = start_date_private, end = end_date_private)[['Close']].reset_index() 
    y_private00.loc[i,:]=list(sample.Close)
y_private00.head()

In [ ]:
y_private00

In [ ]:
model.fit(x_train,y_train)

In [ ]:
prediction11 = model.predict(x_private)
result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction11)
(NMAE(y_private00.reset_index(drop=True),result0)*100).mean()

In [ ]:
y_private00

In [ ]:
train2

In [ ]:
final_dataset

In [ ]:
result0

#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(random_state=123)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill') #나스닥 추가
    train1=pd.concat([train1,kospi['kospi']],axis=1) #코스피 추가
    train1=pd.concat([train1,kosdaq['kosdaq']],axis=1) #코스닥 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

### 최적의 파라미터를 찾기 위해 gridsearch를 돌려보고 최종모델에 반영

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters={'max_features': [4,8,12,20,24], 'n_estimators': [50,100,200],'random_state': [10101]}
grid_reg=GridSearchCV(RandomForestRegressor(),param_grid=parameters,cv=5,scoring = 'neg_mean_squared_error')
grid_reg.fit(x_train.values,
             y_train.values.ravel())
print('GridSearchCV 최적 파라미터:', grid_reg.best_params_)

In [ ]:
scores_df=pd.DataFrame(grid_reg.cv_results_)
scores_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score',
          'split3_test_score','split4_test_score']]

# 찾은 최적 파라미터로 돌려보기

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(max_features=2,n_estimators=300, random_state=101011)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill') #나스닥 추가
    train1=pd.concat([train1,kospi['kospi']],axis=1) #코스피 추가
    train1=pd.concat([train1,kosdaq['kosdaq']],axis=1) #코스닥 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

In [ ]:
result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction)
(NMAE(y_private.reset_index(drop=True),result0)*100).mean()

In [ ]:
parameters={'max_features': [2,4,5], 'n_estimators': [100,200,300],'random_state': [10101]}
grid_reg=GridSearchCV(RandomForestRegressor(),param_grid=parameters,cv=3,scoring = 'neg_mean_squared_error')
grid_reg.fit(x_train.values,
             y_train.values.ravel())
print('GridSearchCV 최적 파라미터:', grid_reg.best_params_)

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
plt.figure(figsize=(12,8))
importances = model.feature_importances_
sorted_index = np.argsort(importances)[::-1]
x_values = range(len(importances))
labels = np.array(list(x_private.columns))[sorted_index]
plt.barh(x_values, importances[sorted_index], tick_label=labels)
plt.title('n_estimator=300 max_features=5')
plt.savefig('g3.png')

# 코스피만 해볼까? 코스닥지표는 빼보자

In [141]:
#자체적으로 결과값과 예측값을 비교하기위해 결과셋 생성 
start_date_private='20211129' 
end_date_private='20211203' 
y_private1=pd.DataFrame(columns=[0,1,2,3,4]) 
for i in tqdm(list(kospi_list.종목코드)):
    sample = fdr.DataReader(i, start = start_date_private, end = end_date_private)[['Close']].reset_index() 
    y_private1.loc[i,:]=list(sample.Close)
y_private1.head()

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:23<00:00,  8.51it/s]


,0,1,2,3,4
000070,95900,91100,95700,98000,98900
000080,30300,29000,29400,29550,30600
000100,56369,55317,55221,57518,57709
000120,129000,124000,125000,129000,131000
000150,110500,108500,112000,110000,108500


In [142]:
from sklearn.ensemble import RandomForestRegressor
#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(max_features=5,n_estimators=300, random_state=10101)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(kospi_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,kospi['kospi']],axis=1) #코스피 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction)
(NMAE(y_private1.reset_index(drop=True),result0)*100).mean()

  0%|                                                                                          | 0/196 [00:00<?, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

  4%|██▉                                                                               | 7/196 [00:01<00:49,  3.80it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

  7%|█████▊                                                                           | 14/196 [00:03<00:45,  3.96it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

 11%|████████▋                                                                        | 21/196 [00:05<00:52,  3.33it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

 14%|███████████▌                                                                     | 28/196 [00:07<00:44,  3.78it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

...

 88%|██████████████████████████████████████████████████████████████████████▌         | 173/196 [00:45<00:07,  3.15it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

 92%|█████████████████████████████████████████████████████████████████████████▍      | 180/196 [00:47<00:03,  4.03it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 95%|████████████████████████████████████████████████████████████████████████████▎   | 187/196 [00:49<00:01,  4.92it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: Set

 98%|██████████████████████████████████████████████████████████████████████████████▊ | 193/196 [00:50<00:00,  4.13it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3707110114.py:25: SettingWithCopyWarning: 
A 

4.944902867477171

평가점수

In [143]:
import sklearn
from sklearn.model_selection import KFold

df_shuffled=sklearn.utils.shuffle(final_dataset).reset_index(drop=True) #최종데이터의 행을 섞는다
df_shuffled
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5) #crossvalidation 할때 몇개로 쪼갤것인가

model=RandomForestRegressor(random_state= 4424000) #모델의 하이퍼 파라미터 조정

x_train = df_shuffled.drop(['Change','after5days'],axis=1)            #훈련 데이터셋 생성       
y_train = df_shuffled[['after5days']]

for train_index, test_index  in kfold.split(df_shuffled):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1))) #5개 셋의 점수를 저장
np.mean(NMAEscore) #평균

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usin

0.024189687447283732

In [144]:
import sklearn
from sklearn.model_selection import KFold

df_shuffled=sklearn.utils.shuffle(final_dataset).reset_index(drop=True) #최종데이터의 행을 섞는다
df_shuffled
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5) #crossvalidation 할때 몇개로 쪼갤것인가

model=RandomForestRegressor(max_features=5,n_estimators=300,random_state= 121011) #모델의 하이퍼 파라미터 조정

x_train = df_shuffled.drop(['Change','after5days'],axis=1)            #훈련 데이터셋 생성       
y_train = df_shuffled[['after5days']]

for train_index, test_index  in kfold.split(df_shuffled):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1))) #5개 셋의 점수를 저장
np.mean(NMAEscore) #평균

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1166340683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1166340683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1166340683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1166340683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usin

0.02434691901104732

In [ ]:
plt.figure(figsize=(12,8))
importances = model.feature_importances_
sorted_index = np.argsort(importances)[::-1]
x_values = range(len(importances))
labels = np.array(list(x_private.columns))[sorted_index]
plt.barh(x_values, importances[sorted_index], tick_label=labels)
plt.title('KOSPI : max_features=5 n_estimators=300')
plt.savefig('g8.png')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(random_state=11234)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(kospi_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,kospi['kospi']],axis=1) #코스피 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction)
(NMAE(y_private1.reset_index(drop=True),result0)*100).mean()

In [ ]:
plt.figure(figsize=(12,8))
importances = model.feature_importances_
sorted_index = np.argsort(importances)[::-1]
x_values = range(len(importances))
labels = np.array(list(x_private.columns))[sorted_index]
plt.barh(x_values, importances[sorted_index], tick_label=labels)
plt.title('KOSPI : Default')
plt.savefig('g5.png')

# 코스닥만

In [146]:
#자체적으로 결과값과 예측값을 비교하기위해 결과셋 생성 
start_date_private='20211129' 
end_date_private='20211203' 
y_private2=pd.DataFrame(columns=[0,1,2,3,4]) 
for i in tqdm(list(kosdaq_list.종목코드)):
    sample = fdr.DataReader(i, start = start_date_private, end = end_date_private)[['Close']].reset_index() 
    y_private2.loc[i,:]=list(sample.Close)
y_private2.head()

100%|████████████████████████████████████████████████████████████████████████████████| 147/147 [00:15<00:00,  9.79it/s]


,0,1,2,3,4
000250,43250,42150,42750,43100,44900
003380,9320,9020,9180,9330,9480
005290,35000,33650,34100,34100,33850
006730,8577,8312,8488,8704,8900
007390,18100,17700,17800,18300,18600


In [147]:
from sklearn.ensemble import RandomForestRegressor
#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(max_features=5, n_estimators=300, random_state=12121)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(kosdaq_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill') #나스닥 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.concat([train1,kosdaq['kosdaq']],axis=1) #코스닥 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction)
(NMAE(y_private2.reset_index(drop=True),result0)*100).mean()

  0%|                                                                                          | 0/147 [00:00<?, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

  5%|███▉                                                                              | 7/147 [00:01<00:38,  3.64it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

 10%|███████▋                                                                         | 14/147 [00:03<00:34,  3.85it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

 14%|███████████▌                                                                     | 21/147 [00:05<00:31,  4.04it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

 19%|███████████████▍                                                                 | 28/147 [00:08<00:39,  3.02it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

...

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 85%|████████████████████████████████████████████████████████████████████            | 125/147 [00:31<00:05,  4.37it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24:

 89%|███████████████████████████████████████████████████████████████████████▎        | 131/147 [00:32<00:03,  4.60it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 94%|███████████████████████████████████████████████████████████████████████████     | 138/147 [00:34<00:01,  4.75it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24:

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 144/147 [00:35<00:00,  4.70it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/786499039.py:25: SettingWithCopyWarning: 
A val

5.091345489380133

In [ ]:
plt.figure(figsize=(12,8))
importances = model.feature_importances_
sorted_index = np.argsort(importances)[::-1]
x_values = range(len(importances))
labels = np.array(list(x_private.columns))[sorted_index]
plt.barh(x_values, importances[sorted_index], tick_label=labels)
plt.title('KOSDAQ : max_features=5 n_estimators=300')
plt.savefig('g6.png')

In [ ]:
R2 = r2_score(y_private2.reset_index(drop=True), result0)
R2

In [ ]:
prediction2 = model.predict(x_train)

In [ ]:
result_train=pd.DataFrame(columns=[0])
for i in range(0,1):
    result_train[i]=list(prediction2)
result_train

In [ ]:
R2_train = r2_score(y_train.reset_index(drop=True), result_train)
R2_train

In [ ]:
plt.scatter(x_train,y_train)
plt.plot(x_train,prediction2)

In [145]:
from sklearn.ensemble import RandomForestRegressor
#12월 첫째주 private 예측하기 
start_date='20201224'
end_date='20211129'
model=RandomForestRegressor(random_state=12222)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(kosdaq_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill') #나스닥 추가
    train1=pd.concat([train1,kosdaq['kosdaq']],axis=1) #코스닥 추가
    train1['sector']=new_stock_list.loc[i][0] #섹터 클러스터 더미변수 추가
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    train2=train1.dropna()

    final_dataset=pd.concat([final_dataset,train2], ignore_index=True) #최종 데이터셋 생성
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

result0=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result0[i]=list(prediction)
(NMAE(y_private2.reset_index(drop=True),result0)*100).mean()

  0%|                                                                                          | 0/147 [00:00<?, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A 

  5%|███▉                                                                              | 7/147 [00:01<00:35,  3.91it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A 

 10%|███████▋                                                                         | 14/147 [00:03<00:44,  2.98it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A 

 14%|███████████▌                                                                     | 21/147 [00:05<00:30,  4.11it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A 

 19%|███████████████▍                                                                 | 28/147 [00:07<00:28,  4.20it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A 

...

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 85%|████████████████████████████████████████████████████████████████████            | 125/147 [00:29<00:04,  4.87it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: Set

 89%|███████████████████████████████████████████████████████████████████████▎        | 131/147 [00:30<00:03,  4.74it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A 

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['5avg']=list2 #직전 5일간 평균가격
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 94%|███████████████████████████████████████████████████████████████████████████     | 138/147 [00:31<00:01,  5.14it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: Set

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
 98%|██████████████████████████████████████████████████████████████████████████████▎ | 144/147 [00:32<00:00,  5.28it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['last5days']=list1 #직전 5일간 평균 변화율
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/2941385584.py:

NameError: name 'y_private2' is not defined

In [148]:
import sklearn
from sklearn.model_selection import KFold

df_shuffled=sklearn.utils.shuffle(final_dataset).reset_index(drop=True) #최종데이터의 행을 섞는다
df_shuffled
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5) #crossvalidation 할때 몇개로 쪼갤것인가

model=RandomForestRegressor(random_state= 4424000) #모델의 하이퍼 파라미터 조정

x_train = df_shuffled.drop(['Change','after5days'],axis=1)            #훈련 데이터셋 생성       
y_train = df_shuffled[['after5days']]

for train_index, test_index  in tqdm(kfold.split(df_shuffled)):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1))) #5개 셋의 점수를 저장
np.mean(NMAEscore) #평균

0it [00:00, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3496970683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
1it [00:15, 15.86s/it]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3496970683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
2it [00:32, 16.10s/it]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3496970683.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
3it [00:46, 15.52s/it]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3496970683.py:20: DataConversionWarning: A column-vector y was passed when a 1

0.028310381305329042

In [149]:
import sklearn
from sklearn.model_selection import KFold

df_shuffled=sklearn.utils.shuffle(final_dataset).reset_index(drop=True) #최종데이터의 행을 섞는다
df_shuffled
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5) #crossvalidation 할때 몇개로 쪼갤것인가

model=RandomForestRegressor(max_features=5,n_estimators=300,random_state= 121011) #모델의 하이퍼 파라미터 조정

x_train = df_shuffled.drop(['Change','after5days'],axis=1)            #훈련 데이터셋 생성       
y_train = df_shuffled[['after5days']]

for train_index, test_index  in tqdm(kfold.split(df_shuffled)):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1))) #5개 셋의 점수를 저장
np.mean(NMAEscore) #평균

0it [00:00, ?it/s]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3476071019.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
1it [00:19, 19.24s/it]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3476071019.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
2it [00:34, 17.08s/it]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3476071019.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
3it [00:50, 16.45s/it]C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3476071019.py:20: DataConversionWarning: A column-vector y was passed when a 1

0.02688931573510951

In [ ]:
plt.figure(figsize=(12,8))
importances = model.feature_importances_
sorted_index = np.argsort(importances)[::-1]
x_values = range(len(importances))
labels = np.array(list(x_private.columns))[sorted_index]
plt.barh(x_values, importances[sorted_index], tick_label=labels)
plt.title('KOSDAQ : Default')
plt.savefig('g10.png')

# 1 데이터의 개수를 늘려보기

#나스닥 코스피 지수 추출
start_date='20161224'
end_date='20211129'
nasdaq=pdr.get_data_yahoo('^IXIC',start_date,end_date)
nasdaq['nasdaq']=nasdaq['Close']
kospi=pdr.get_data_yahoo('^KS11',start_date,end_date)

#12월 첫째주 private 예측하기 
start_date='20161224'
end_date='20211129'
model=RandomForestRegressor(max_features=6,n_estimators=100, random_state=10101)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()

#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
 
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 nan추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,nasdaq['Adj Close']],axis=1)
    train1['nasdaq']=train1['Adj Close'].fillna(method='ffill')
    train1['nasdaq_change']=train1.nasdaq.pct_change(4) #직전5일간 나스닥 변화율
    train1['kospi']=kospi['Close']
    train1['kosdaq']=kosdaq['Close']
    train2=train1.dropna()
    del train2['Adj Close']
    final_dataset=pd.concat([final_dataset,train2], ignore_index=True)
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days','Adj Close'],axis=1)
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

In [ ]:
result1=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result1[i]=list(prediction)
(NMAE(y_private.reset_index(drop=True),result1)*100).mean()

# 클러스터 변수를 넣은 모델링

#12월 첫째주 private 예측하기 
start_date='20161224'
end_date='20211129'
predictions = pd.DataFrame(columns=[0,1,2,3,4])
model=RandomForestRegressor(random_state= 123)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()
kospi['kospi']=kospi.Close
#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
     
    #데이터 불러오기
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 0으로 추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,kospi['kospi']],axis=1)
    train1['sector']=new_stock_list.loc[i][0]
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    
    train2=train1.dropna()
    final_dataset=pd.concat([final_dataset,train2], ignore_index=True)
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1) 
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset
x_train = x_train.drop(['Change','after5days'],axis=1)                      
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

In [ ]:
result=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    result[i]=list(prediction)
result
(NMAE(y_private.reset_index(drop=True),result)*100).mean()

# 나스닥을 넣은 클러스터

In [ ]:
#12월 첫째주 private 예측하기 
start_date='20161224'
end_date='20211129'
predictions = pd.DataFrame(columns=[0,1,2,3,4])
model=RandomForestRegressor(n_estimators=100, random_state= 123)
final_dataset=pd.DataFrame()
x_private=pd.DataFrame()
kospi['kospi']=kospi.Close
#전체수익률예측코드
for i in tqdm(list(stock_list.종목코드)):
     
    #데이터 불러오기
    sample = fdr.DataReader(i, start = start_date, end = end_date).reset_index()
    train=sample[['Date','Close','Change']]
    
    # 평균변화율 변수 생성
    list1=[np.nan,np.nan,np.nan,np.nan] #앞에 4일은 직전5일간 평균 변화율을 계산할수 없기에 0으로 추가
    list2=[np.nan,np.nan,np.nan,np.nan]
    j=0
    for k in range(4,len(train)):
        list1.append(train.Change[j:k+1].mean())
        list2.append(train.Close[j:k+1].mean())
        j+=1
    train['last5days']=list1 #직전 5일간 평균 변화율
    train['5avg']=list2 #직전 5일간 평균가격
    train['after5days']=train['5avg'].shift(-5) #기준일 대비 5일 이후의 평균가격
    
    #한국시장과 미장을 합치기 위해 인덱스 변경
    train.set_index('Date',inplace=True)
    train1=pd.concat([train,kospi['kospi']],axis=1)
    train1=pd.concat([train,nasdaq['nasdaq']],axis=1)
    train1['nasdaq']=train1['nasdaq'].fillna(method='ffill')
    train1['sector']=new_stock_list.loc[i][0]
    train1=pd.get_dummies(data=train1,columns=['sector'], prefix = 'sector')
    
    train2=train1.dropna()
    final_dataset=pd.concat([final_dataset,train2], ignore_index=True)
    final_dataset=final_dataset.fillna(0)

    #private set 생성
    x_private = pd.concat([x_private,train1[-1:]], ignore_index=True)
x_private=x_private.drop(['Change','after5days'],axis=1) 
x_private=x_private.fillna(0)

#test set 생성
x_train = final_dataset
x_train = x_train.drop(['Change','after5days'],axis=1)                      
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

In [ ]:
last_result=pd.DataFrame(columns=[0,1,2,3,4])
for i in range(0,5):
    last_result[i]=list(prediction)
last_result

In [ ]:
(NMAE(y_private.reset_index(drop=True),last_result)*100).mean()

In [ ]:
parameters={'max_depth':[2,3],'max_features': [20,25], 'n_estimators': [100,200,500],'random_state': [101]}
grid_reg=GridSearchCV(RandomForestRegressor(),param_grid=parameters,cv=5,scoring = 'neg_mean_squared_error')
grid_reg.fit(x_train.values,
             y_train.values.ravel())
print('GridSearchCV 최적 파라미터:', grid_reg.best_params_)

# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_reg.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(x_private)

In [112]:
from sklearn.model_selection import KFold
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5)
model=RandomForestRegressor(random_state= 119)
for train_index, test_index  in kfold.split(x_train):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1)))
NMAEscore

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369829855.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369829855.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369829855.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369829855.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usin

AttributeError: 'list' object has no attribute 'mean'

In [115]:
np.mean(NMAEscore)

0.03378555338254949

NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5)
model=RandomForestRegressor(max_features=5,n_estimators=300,random_state= 120)
for train_index, test_index  in kfold.split(x_train):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1)))
NMAEscore.mean()
#평균3.4

In [121]:
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5)
model=RandomForestRegressor(max_features=5,n_estimators=300,random_state= 12100)
for train_index, test_index  in kfold.split(x_train):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1)))
np.mean(NMAEscore)

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369758992.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369758992.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369758992.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3369758992.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usin

0.09127332579400321

In [124]:
NMAEscore

[0.1029162172290968,
 0.15394159947958208,
 0.08917780526202489,
 0.05760559301570427,
 0.052725413983607985]

In [ ]:
#섞어보자

In [140]:
import sklearn
from sklearn.model_selection import KFold

df_shuffled=sklearn.utils.shuffle(final_dataset).reset_index(drop=True) #최종데이터의 행을 섞는다
df_shuffled
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5) #crossvalidation 할때 몇개로 쪼갤것인가

model=RandomForestRegressor(random_state= 4424000) #모델의 하이퍼 파라미터 조정

x_train = df_shuffled.drop(['Change','after5days'],axis=1)            #훈련 데이터셋 생성       
y_train = df_shuffled[['after5days']]

for train_index, test_index  in kfold.split(df_shuffled):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1))) #5개 셋의 점수를 저장
np.mean(NMAEscore) #평균

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/3755439424.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usin

0.025432587417720764

In [138]:
import sklearn
df_shuffled=sklearn.utils.shuffle(final_dataset).reset_index(drop=True)
df_shuffled
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5)
model=RandomForestRegressor(max_features=5,n_estimators=300,random_state= 12100)
x_train = df_shuffled.drop(['Change','after5days'],axis=1)                   
y_train = df_shuffled[['after5days']]
for train_index, test_index  in kfold.split(df_shuffled):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1)))
np.mean(NMAEscore)

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/579914980.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/579914980.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/579914980.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/579914980.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ra

0.029987433927862156

In [139]:
NMAEscore

[0.029637733418658958,
 0.029968851531822688,
 0.03047081402965738,
 0.030487422728201086,
 0.029372347930970665]

In [130]:
#test set 생성
x_train = final_dataset.drop(['Change','after5days'],axis=1)                   
y_train = final_dataset[['after5days']]
    
#예측
model.fit(x_train,y_train)
prediction = model.predict(x_private)

,Close,last5days,5avg,nasdaq,kospi,kosdaq,sector_필수소비재,sector_건강관리그룹1,sector_산업재그룹0,sector_산업재그룹2,...,sector_건강관리그룹0,sector_소재그룹1,sector_IT그룹0,sector_경기관련소비재그룹1,sector_에너지,sector_금융그룹0,sector_소재그룹2,sector_유틸리티,sector_커뮤니케이션서비스,sector_건강관리그룹2
51803,10300.0,0.006541,10140.0,13614.509766,3247.429932,990.190002,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70646,149000.0,-0.007029,153660.0,14007.700195,3084.669922,957.849976,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,13350.0,0.014126,12980.0,14448.580078,3068.820068,1003.270020,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23705,23700.0,0.033750,22430.0,15259.240234,3199.270020,1038.329956,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53221,393000.0,-0.003264,394600.0,15235.839844,3025.489990,1008.950012,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5023,5130.0,-0.008338,5236.0,14512.440430,3060.270020,1001.460022,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56780,12150.0,0.001325,11940.0,13403.389648,3056.530029,956.919983,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8797,40000.0,0.000494,40940.0,15993.709961,2947.379883,1032.770020,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47131,42750.0,0.000764,42890.0,13215.240234,3039.530029,952.109985,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5)
model=RandomForestRegressor(max_features=24,n_estimators=100,random_state= 122)
for train_index, test_index  in kfold.split(x_train):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1)))
np.mean(NMAEscore)

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1420794893.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1420794893.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1420794893.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/1420794893.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usin

0.034240122162164086

In [118]:
NMAEscore=[]
n_iter=0
kfold = KFold(n_splits=5)
model=RandomForestRegressor(max_features=2,n_estimators=300,random_state= 123)
for train_index, test_index  in kfold.split(x_train):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    small_x_train, val_x_train = x_train.loc[list(train_index),], x_train.loc[list(test_index),]
    small_y_train, val_y_train = y_train.loc[list(train_index),], y_train.loc[list(test_index),]
    #학습 및 예측 
    model.fit(small_x_train,small_y_train) 
    pred2=model.predict(val_x_train)
    n_iter += 1
    NMAEscore.append(NMAE(val_y_train.to_numpy(),pred2.reshape(len(val_y_train),1)))
np.mean(NMAEscore)

C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/162285958.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/162285958.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/162285958.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(small_x_train,small_y_train)
C:\Users\kimpu\AppData\Local\Temp/ipykernel_8380/162285958.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ra

0.16442444220518465